In [1]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Sequential
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [29]:
#----Create image augmentation generator
datagen = ImageDataGenerator(rotation_range = 90,
                            horizontal_flip = True,
                            vertical_flip = True,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            shear_range = 0.2,
                            zoom_range = 0.3,
                            brightness_range = (0.5,1))

In [ ]:
foo

In [30]:
def data_aug(parent_folder_path, dataset, sub_folder):
    
    #----Getting list of file names from sub_folder
    img_name_list = os.listdir(os.path.join(parent_folder_path, dataset, sub_folder))
    number_of_images_to_generate = (1000//len(img_name_list)) + 1
    
    #----Create new aug_{sub_folder} next to original sub_folder for saving augmented images,
        #if directory already exist then no action will be done
    # os.makedirs(os.path.join(parent_folder_path, dataset, sub_folder, f"aug_{sub_folder}"), exist_ok=True)
    
    #----Set save path = aug_{sub_folder} 
    save_to_dir = os.path.join(parent_folder_path, dataset, sub_folder)
    
    #----Remove hidden file(s) in img_name_list
    if "." in img_name_list:
        img_name_list.remove(".")
    if ".." in img_name_list:
        img_name_list.remove("..")
    if ".ipynb_checkpoints" in img_name_list:
        img_name_list.remove(".ipynb_checkpoints")

    #----lopping through each image file...
    for original_img_name in img_name_list:

        #----load the image
        original_img_file_path = os.path.join(parent_folder_path, dataset, sub_folder, original_img_name) 
        loading_img = load_img(original_img_file_path)

        #----convert image to 3D array
        img_3d = img_to_array(loading_img)

        #----convert 3d image to 4d (a list with one 3d image)
        img_list = np.expand_dims(img_3d, axis=0)
        
        
        #(can be replaced)----Create image iterator 
        image_iterator = datagen.flow(img_list, batch_size=1
                                  ,save_to_dir=save_to_dir, save_prefix=f'aug_{original_img_name[0:-4]}'
                                      , save_format='jpg')

        #(can be replaced)----Save image
        for i in range(number_of_images_to_generate):
            image_iterator.next()
            

In [37]:
############----SET INPUTS----################

#----Set original image folder path, dataset to use, etc
parent_folder_path = "../../data"
dataset_list = ["test", "train", "val"]
sub_folder = "beetroot"

#Set number of augmented images to generate
o
number_of_images_to_generate = 1000/

In [38]:
#----Calling function
for dataset in dataset_list:
    data_aug(parent_folder_path, dataset, sub_folder)


In [11]:
#----EXTRA CODE FOR VIEWING THE AUGMENTED IMAGES, SEE LINES WITH "(can be replaced)"

# image_iterator = datagen.flow(img_list, batch_size=1)

# rows = 2
# columns = 2

# fig, axes = plt.subplots(rows, columns)
# fig.set_size_inches(10,10)

# for r in range(rows):
#     for c in range(columns):
#         image_batch = image_iterator.next()
#         image = image_batch[0].astype('uint8')
#         axes[r,c].imshow(image)